In [1]:
import pandas as pd 
import ipywidgets as widgets
import datetime
import sqlalchemy
import psycopg2
import golf_functions

This notebook is to add golf course tee infromation. Tee information will be stored in the course_tee and hole_info table.

To enter new course tee information you will need:
- par
- tee
- total_distance
- F9_distance
- B9_distance
- rating
- slope
- for each hole
    - par
    - distance
    - handicap

In [ ]:
course_select_query = '''
Select cn.course_id
    ,cn.course_name
    ,cv.course_version_id
from "Courses"."course_name" as cn
left join (select course_id, MAX(course_version_id) as course_version_id from "Courses"."course_version" group by course_id) as cv on cn.course_id = cast(cv.course_id as int)
'''

In [ ]:
courses = golf_functions.pull_from_sql(course_select_query)

In [ ]:
course_list = list(courses['course_name'])
course_list.sort()
course_select = widgets.Dropdown(options = course_list)
course_select

In [ ]:
course_info_entry = courses[courses['course_name'] == course_select.value]

In [ ]:
par_lbl = widgets.Label('Par')
par_ent = widgets.Text()
tee_lbl = widgets.Label('Tee Name')
tee_ent = widgets.Text()
total_distance_lbl = widgets.Label('Total Distance')
total_distance_ent = widgets.IntText()
F9_distance_lbl = widgets.Label('Front 9 Distance')
F9_disance_ent = widgets.IntText()
B9_distance_lbl = widgets.Label('Back 9 Distance')
B9_distance_ent = widgets.IntText()
rating_lbl = widgets.Label('Rating')
rating_ent = widgets.FloatText()
slope_lbl = widgets.Label('Slope')
slope_ent = widgets.IntText()

In [ ]:
display(par_lbl)
display(par_ent)
display(tee_lbl)
display(tee_ent)
display(total_distance_lbl)
display(total_distance_ent)
display(F9_distance_lbl)
display(F9_disance_ent)
display(B9_distance_lbl)
display(B9_distance_ent)
display(rating_lbl)
display(rating_ent)
display(slope_lbl)
display(slope_ent)

In [ ]:
course_info_dict = {
    'total_par':par_ent.value,
    'tee':tee_ent.value,
    'total_distance':total_distance_ent.value,
    'f9_distance':F9_disance_ent.value,
    'b9_distance':B9_distance_ent.value,
    'rating':rating_ent.value,
    'slope':slope_ent.value,
    'course_name':course_select.value
}
df = pd.DataFrame([course_info_dict],columns=course_info_dict.keys())
course_tee_df = pd.merge(course_info_entry,df, on='course_name')
print('Is the correct course information')
course_tee_df

If the above course information is correct please run the next cell. 

In [ ]:
num_holes = widgets.IntText()
num_holes

In [ ]:
for h in range(1,num_holes.value + 1):
    globals()['par_%s' % h + '_lbl'] = widgets.Label('par')
    globals()['par_%s' % h + '_ent'] = widgets.Text()
    globals()['distance_%s' % h + '_lbl'] = widgets.Label('distance')
    globals()['distance_%s' % h + '_ent'] = widgets.IntText()
    globals()['handicap_%s' % h + '_lbl'] = widgets.Label('Course Handicap')
    globals()['handicap_%s' % h + '_ent'] = widgets.Text()
    display(widgets.Label('For Hole - ' + str(h)))
    display(globals()['par_%s' % h + '_lbl'])
    display(globals()['par_%s' % h + '_ent'])
    display(globals()['distance_%s' % h + '_lbl'])
    display(globals()['distance_%s' % h + '_ent'])
    display(globals()['handicap_%s' % h + '_lbl'])
    display(globals()['handicap_%s' % h + '_ent'])

In [ ]:
hole_list = []
par_list = []
distance_list = []
handicap_list = []
for h in range(1,num_holes.value + 1):
    hole_list.append(str(h))
    par_list.append(globals().get('par_%s' % h + '_ent').value)
    distance_list.append(globals().get('distance_%s' % h + '_ent').value)
    handicap_list.append(globals().get('handicap_%s' % h + '_ent').value)

hole_info_dict = {
    'hole':hole_list,
    'par':par_list,
    'distance':distance_list,
    'handicap':handicap_list
}
hole_info = pd.DataFrame(hole_info_dict,columns=hole_info_dict.keys())

In [ ]:
course_tee_df

In [ ]:
hole_info

If the above information is correct run the cell below

In [ ]:
schema = '"Courses"'
procedure = 'add_new_course_tee'
for h in range(0,hole_info.shape[0]):
    values = (
        course_tee_df["course_id"][0],
        course_tee_df["total_par"][0],
        course_tee_df["tee"][0],
        course_tee_df["total_distance"][0],
        course_tee_df["f9_distance"][0],
        course_tee_df["b9_distance"][0],
        course_tee_df["rating"][0],
        course_tee_df["slope"][0],
        course_tee_df["course_version_id"][0],
        hole_info["hole"][h],
        hole_info["par"][h],
        hole_info["distance"][h],
        hole_info["handicap"][h]
    )
    golf_functions.load_procedure(schema, procedure, values)